# II Cryptanalyse de RSA

## 16

In [33]:
Pr.<x,y,u> = PolynomialRing(ZZ)

In [34]:
def gen(k,gamma):
    # On prend au hasard p le premier nombre premier.
    p = random_prime(2**k,lbound = 2**(k-1), proof=false)
    
    # On prend au hasard q le deuxième nombre premier.  
    q = random_prime(2**k,lbound = 2**(k-1), proof=false)
    
    # Si P et Q sont égal, on recherche pour une autre valeur qui est
    # premier.
    if q==p:
        q=next_prime(q)
        
    N = p*q
    
    # On calcule phi(N)= (p-1)*(q-1)
    phi =(p-1)*(q-1)
    
    # On calcule d'abord d
    d = floor(N**gamma)
    
    # On cherche le plus grand d possible inferieur à N^gamma
    # et qui soit premier avec phi
    while gcd(phi,d) != 1:
        d = d-1
        
    # On calcule e=d^(-1)mod(phi)
    e = d.inverse_mod(phi)
    return N,e,d

In [24]:
gen(2048,0.5)

(643022092392819660182994303434230600469863848162575436835615212823598844534917513839036206796181702530474289870016670503713210893663504412010474779064868172619306303578872195010835989911931608901037167421869951050575310072792642458438422326694697552574531971265119592520697067060445151331225310558224270241983527235175196600609623977405666301296678240029386092194873110366433616916327882914216846070263360205175925041210113631440014188520432108558362632454163143844816792467646811817873963707835176125388666441957285846618819909740246488539397615126158639559633694811845127351599297869232649429848506977716606095408355099321769691072322886377301646418076230985517018331783126806301750499274861772125494937026750585150308266646931705012698184287911340805710443023005577034999309948434984917123594275398162845863904350042461410892293812260505745584516083080568637932313303369132112080850285592626837425606678184568174117165603946084117134168540206179155930638410072859419024804256084227952359409249011

## 21

In [25]:
def changevar1e(f):
    
    # On recupère le degrée du polynômes pour créer les listes de monômes.
    d = f.degree()
    
    # On recherche dans la fonction tous les coefficient des mônomes de la forme x**i*y**j avec i et j entre 1 et d
    for i in range(1,d):
        for j in range(1,d):
        # On calcule tous les coefficients de chaque monôme de la forme x**i*y**j
            k = f.coefficient(x**i*y**j)
            if i < j:
                # Comme la puissance de x est supérieur à la puissance de y.
                f = f  - k*((x**i*y**j) - ((u-1)**i*y**(j-i)))
             
            if i ==j :
                #Comme la puissance de x est égale à la puissance de y.
                f = f   - k*((x**i*y**j) - (u-1)**i)
            
            if j<i :
                # Comme la puissance de y est supérieur à la puissance de x.
                f = f - k*((x**i*y**j) -  ((u-1)**j*x**(i-j)))
            
    return f

In [26]:
def lattice(m,t,N,e,gamma):
    e1 = ZZ(e)
    # D'après l'énoncé:
    X = integer_ceil(e1**gamma)
    Y = integer_ceil(e1**(0.5))
    U = 1+X*Y
    
    # On crée la fonction ft
    
    ft = u + (N+1)*x
    
    # On crée la liste de famille de polynomes ht
    H = [y**j*ft**k*e**(m-k) for j in range(1,t+1) for k in [integer_floor(m/t)*j..m]]
    
    # Pour chaque polynôme ht on fait le changement de x*y par (u-1)
    H1 = [changevar1e(i) for i in H]
   
    # On crée la liste des deux familles de pôlynomes.
    L =[x**i*ft**k*e**(m-k) for k in range(m+1) for i in range(m-k+1)]+H1 
    
    # On rajoute x*X,y*Y,u*U
    L1 = [i.subs({x:x*X, y:y*Y, u:u*U}) for i in L]
    
    # On crée la liste des monômes.
    M =[x**(k-i)*u**i for k in range(m+1) for i in range(k+1)]+[y**j*u**k for j in range(1,t+1) for k in [integer_floor(m/t)*j..m]]
    
    # On crée la matrice avec les coefficients de chaques monômes par rapport aux polynômes.
    Mat = matrix(ZZ,len(L1),len(M))
    for i in range(len(L1)):
        for j in range(len(M)):
            Mat[i,j] = L1[i].monomial_coefficient(M[j])
    return Mat

In [27]:
# On essaie avec un petit exemple
N,e,d =gen(5,0.7)
print("N = %d e =%d" %(N, e))
gamma = 0.7
X = integer_ceil(e**gamma)
Y = integer_ceil(e**(0.5))
U = 1+X*Y
print("X = %d  Y = %d  U = %d" %(X, Y, U))

N = 899 e =617
X = 90  Y = 25  U = 2251


In [28]:
lattice(2,1,N,e,gamma)

[      380689            0            0            0            0            0            0]
[           0     34262010            0            0            0            0            0]
[           0            0            0   3083580900            0            0            0]
[           0     49977000      1388867            0            0            0            0]
[           0            0            0   4497930000    124998030            0            0]
[           0            0            0   6561000000    364662000      5067001            0]
[           0    -72900000     -4051800            0 164097900000   9120601800    126675025]

## 22

In [29]:
def changevar2(f):
    #On change u en 1+x*y 
    f = f.substitute(u = 1+x*y)
    return f

## 24

In [12]:
def factorisation(N,e,gamma,m):
    e1 = ZZ(e)
   # D'après l'énoncé:
    X = integer_ceil(e1**gamma)
    Y = integer_ceil(e1**(0.5))
    U = 1+X*Y
    t = integer_floor(m*(1-2*gamma))
    Mat =lattice(m,t,N,e,gamma)
    Matl = Mat.LLL()
    
    Pr.<x,y,u> = PolynomialRing(QQ)
    # On crée une liste de monômes
    M =[(x/X)**(k-i)*(u/U)**i for k in range(m+1) for i in range(k+1)]+[(y/Y)**j*(u/U)**k for j in range(1,t+1) for k in [integer_floor(m/t)*j..m]]
    
    # On crée les polynomes de chaque ligne de la matrice.
    P=[sum(Matl[i][j]*M[j] for j in range(len(M))) for i in range(len(M))]
    
    
    
    # On remplace u par 1+x*y
    Pchange = [changevar2(i) for i in P]
    
    
   
    # on initialise i=0
    i = 0
    P1 = Pchange[i]
    P2 = Pchange[i+1]
    
    # On met Pxy un nouvelle anneau polynomial en fonction de x et y
    Pxy.<x, y> = PolynomialRing(ZZ)
    # On caste P1 et P2 en Pxy
    
    PP1 = Pxy(P1)
    PP2 = Pxy(P2)
   
    
    res = PP1.resultant(PP2,y)
    
     # On met Px un nouvelle anneau polynomial en fonction de x
    Px.<x> = PolynomialRing(ZZ)
    
    resx = Px(res)
    
    
    while res.degree() < 0  and i!=len(M)-2:
        i = i+1
        P1 = Pchange[i]
        P2 = Pchange[i+1] 
        PP1 = Pxy(P1)
        PP2 = Pxy(P2)
        
    
        # On met Px un nouvelle anneau polynomial en fonction de x
   
        res = PP1.resultant(PP2,y)
        
        Px.<x> = PolynomialRing(QQ)
    
        resx = Px(res)
        
        
    
        
    
    # On regarde sur on est sorti de la liste de fonctions
    if i == len(M)-2:
        return (-1)
    # On cherche la premiere valeur   
    x0 = resx.roots(QQ)[0][0]
    
    
    #On reinjecte cette valeur sur le premier polynôme
    PPP1 =P2.subs(x=x0)
    
    # On met Py un nouvelle anneau polynomial en fonction de y
    Py.<y> = PolynomialRing(ZZ)
    
    # On le caste en Py
    equy = Py(PPP1)
    
    # On cherche la racine de cette équation en fonction de y
    y0 =equy.roots(QQ)[0][0]
    
    # D'aprés les résultats de la question 16 et 17
    d = (x0*(N+1+y0)+1)/e1
    
    return (d )   

In [13]:
#premier exemple demandé
N1,e1,d1 =gen(1024,0.2)
gamma = 0.2
m = 2

In [14]:
d= factorisation(N1,e1,gamma,m)

In [15]:
# On voit que c'est le bon d retrouvé
d ==d1

True

In [16]:
d

1741114410480929386621178571294579490225243945228312144161199861755880439458223473085196421199981771426061197566373833736191

In [17]:
d1

1741114410480929386621178571294579490225243945228312144161199861755880439458223473085196421199981771426061197566373833736191

In [30]:
#deuxième exemple demandé
N2,e2,d2 =gen(1024,0.25)
gamma = 0.25
m = 2

In [19]:
d= factorisation(N2,e2,gamma,m)

IndexError: list index out of range

In [32]:
 e = ZZ(e2)
   # D'après l'énoncé:
X = integer_ceil(e**gamma)
Y = integer_ceil(e**(0.5))
U = 1+X*Y
t = integer_floor(m*(1-2*gamma))
Mat =lattice(m,t,N,e2,gamma)
Matl = Mat.LLL()
    
Pr.<x,y,u> = PolynomialRing(QQ)
# On crée une liste de monômes
M =[(x/X)**(k-i)*(u/U)**i for k in range(m+1) for i in range(k+1)]+[(y/Y)**j*(u/U)**k for j in range(1,t+1) for k in [integer_floor(m/t)*j..m]]
    
# On crée les polynomes de chaque ligne de la matrice.
P=[sum(Matl[i][j]*M[j] for j in range(len(M))) for i in range(len(M))]
    
    
    
# On remplace u par 1+x*y
Pchange = [changevar2(i) for i in P]
    
    
   
 #on initialise i=0
i = 2
P1 = Pchange[i]
P2 = Pchange[i+1]
    
# On met Pxy un nouvelle anneau polynomial en fonction de x et y
Pxy.<x, y> = PolynomialRing(ZZ)
# On caste P1 et P2 en Pxy
    
PP1 = Pxy(P1)
PP2 = Pxy(P2)
   
    
res = PP1.resultant(PP2,y)
    
# On met Px un nouvelle anneau polynomial en fonction de x
Px.<x> = PolynomialRing(ZZ)
    
resx = Px(res)
    
    
while res.degree() < 0  and i!=len(M)-2:
    i = i+1
    P1 = Pchange[i]
    P2 = Pchange[i+1] 
    PP1 = Pxy(P1)
    PP2 = Pxy(P2)
        
    
    # On met Px un nouvelle anneau polynomial en fonction de x
   
    res = PP1.resultant(PP2,y)
        
    Px.<x> = PolynomialRing(QQ)
    
    resx = Px(res)
        
        
    
        
    
# On regarde sur on est sorti de la liste de fonctions
if i == len(M)-2:
    print(-1)
# On cherche la premiere valeur   
x0 = resx.roots(QQ)[0][0]
    
    
#On reinjecte cette valeur sur le premier polynôme
PPP1 = P2.subs(x=x0)
    
# On met Py un nouvelle anneau polynomial en fonction de y
Py.<y> = PolynomialRing(ZZ)
    
# On le caste en Py
equy = Py(PPP1)
    
# On cherche la racine de cette équation en fonction de y
y0 =equy.roots(QQ)[0][0]
    
# D'aprés les résultats de la question 16 et 17
d = (x0*(N+1+y0)+1)/e1
    
print("d =", d) 

TypeError: The input degrees must be a dictionary of variables to exponents.

## 25